In [ ]:
# Projeto Aplicado 3 
# Recomendação de Filmes - The Movies Dataset
# O arquivo disponibilizado no Kaggle possui vários CSV, porém os que contém informações relevantes são somente os 3 abaixo


import pandas as pd
import glob

credits = pd.read_csv('tmdb_5000_credits.csv')
movies = pd.read_csv('tmdb_5000_movies.csv', low_memory=False)

%store credits
%store movies

In [ ]:
credits.head()
credits.dtypes
credits['movie_id'].dropna

In [ ]:
movies.head()
movies.dtypes

# Ao executar o código acima, podemos perceber que a coluna ID do dataset Movies é do tipo Object, sendo necessário alterar a mesma para o tipo int64.
# Para isso, devemos realizar a alteração abaixo.

In [ ]:
# Juntando as duas tabelas com base na coluna ID, realizando um merge.

movies = movies.merge(credits, left_on="id", right_on="movie_id")

movies.head()

# Podemos verificar que a coluna _Overview_ descreve o filme.

In [ ]:
movies['overview']

# Já a coluna _"Genres"_ informa o gênero de cada filme. Pode-se notar também que a coluna está no formato _JSON_

In [ ]:
movies['genres'].unique().tolist()

# Realizando algumas análises com base nas informações dos datasets, a começar com o orçamento dos filmes.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

fig, ax = plt.subplots(1, 2, figsize=(14, 6))

# Orçamento
sns.histplot(movies['budget'], bins=30, kde=True, ax=ax[0], color='skyblue')
ax[0].set_title('Distribuição do Orçamento dos Filmes')
ax[0].set_xlabel('Orçamento em milhões US$')
ax[0].set_ylabel('Contagem de Filmes')

# Receita
sns.histplot(movies['revenue'], bins=30, kde=True, ax=ax[1], color='lightgreen')
ax[1].set_title('Distribuição da Receita dos Filmes')
ax[1].set_xlabel('Receita')
ax[1].set_ylabel('Contagem de Filmes')

plt.tight_layout()
plt.show()

* Orçamento: A maioria dos filmes tem um orçamento relativamente baixo, com poucos filmes apresentando orçamentos muito altos. A distribuição é fortemente inclinada para a direita, indicando que filmes com orçamentos extremamente elevados são raros.
* Receita: A distribuição da receita é semelhante à do orçamento, com a maioria dos filmes gerando receitas menores e um número menor de filmes alcançando receitas extremamente altas. A distribuição também é inclinada para a direita.

In [ ]:
# O padrão para todas as colunas contendo informações é JSON, então devemos tratar essas colunas para limpar os dados de uma maneira mais correta para tratamento.

import json

def extrair_generos(cadeia_generos):
    lista_generos = json.loads(cadeia_generos.replace('\'', '"'))
    return [genero['name'] for genero in lista_generos]

todos_generos = movies['genres'].apply(extrair_generos).explode()
contagem_generos = todos_generos.value_counts()

# gerando um gráfico que mostre a contagem de filmes por gênero
plt.figure(figsize=(10, 6))
sns.barplot(y=contagem_generos.index[:15], x=contagem_generos.values[:15], palette='coolwarm')
plt.title('Top 15 Gêneros de Filmes Mais Comuns')
plt.xlabel('Número de Filmes')
plt.ylabel('Gênero')
plt.show()


* Podemos ver que Drama é a categoria com mais filmes, seguido de comédia e trilers

In [ ]:
plt.figure(figsize=(16, 5))

# Popularidade vs. Média de Votos
plt.subplot(1, 2, 1)
sns.scatterplot(data=movies, x='popularity', y='vote_average', color='teal', alpha=0.5)
plt.title('Popularidade vs. Média de Votos')
plt.xlabel('Popularidade')
plt.ylabel('Média de Votos')

# Popularidade vs. Contagem de Votos
plt.subplot(1, 2, 2)
sns.scatterplot(data=movies, x='popularity', y='vote_count', color='coral', alpha=0.5)
plt.title('Popularidade vs. Contagem de Votos')
plt.xlabel('Popularidade')
plt.ylabel('Contagem de Votos')

plt.tight_layout()
plt.show()


* Popularidade vs. Média de Votos: Não parece haver uma relação clara ou forte entre a popularidade de um filme e sua média de votos. Filmes com diferentes níveis de popularidade podem ter uma ampla gama de médias de votos, sugerindo que a popularidade não é diretamente proporcional à qualidade percebida (como indicado pela média de votos).

* Popularidade vs. Contagem de Votos: Existe uma tendência mais perceptível aqui, onde filmes mais populares tendem a ter uma maior contagem de votos. Isso faz sentido, pois podemos esperar que filmes mais populares atraiam mais espectadores e, consequentemente, mais votos.

In [ ]:
#Análise da duração de filmes

plt.figure(figsize=(8, 5))
sns.histplot(movies['runtime'].dropna(), bins=30, kde=True, color='purple')
plt.title('Distribuição da Duração dos Filmes')
plt.xlabel('Duração (Minutos)')
plt.ylabel('Contagem de Filmes')
plt.show()


* A maioria dos filmes tem uma duração entre 90 e 120 minutos, que é típica para longas-metragens.
* A distribuição apresenta uma cauda longa à direita, indicando a presença de alguns filmes com durações significativamente maiores.
* Há um pequeno número de filmes com duração inferior a 60 minutos, o que pode incluir curtas-metragens ou outros formatos especiais.